# Prueba de hiperparametros con modelo LightGBM

<hr>

En este notebook se realiza la prueba de hiperparametros con el modelo LightGBM A través de las métricas obtenidas en el notebook de busqueda de hiperparametros con LightGBM.


Se importan las librerias y herramientas necesarias

In [1]:
import pandas as pd
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
%matplotlib inline 

Importamos los datos

In [2]:
df = pd.read_pickle("data/data.pkl")

- A continuación se definen la variable objetivo y las variables independientes
- Se definen las columnas numéricas y no numéricas
- Se definen las columnas ordinales y categoricas

In [3]:
variable_objetivo = 'naturaleza'
variables_independientes = df.drop(variable_objetivo,axis=1).columns

datos_numericos = df[variables_independientes].select_dtypes([int, float])
col_no_numericas = df[variables_independientes].select_dtypes(include=['category']).columns
col_numericas = datos_numericos.columns

dict_var_ordinales = {
    'grupo_edad': ['0 a 6', '12 a 17', '18 a 28', '29 a 59', '60 y mas', '7  a 11'],
    'ciclo_de_vida':['Primera infancia', 'Infancia', 'Jovenes','Adolescencia','Adultez','Persona Mayor'],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = df[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = df[col_categoricas]

Se importan las librerias necesarias para la creacion del pipeline

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

Se mappean los valores de `grupo_edad` y `ciclo_de_vida` y se crea el pipeline_ordinal y posteriormente el pipeline_procesado con las variables categoricas, el pipeline_ordinal y las columnas ordinales

In [5]:
mapping = [{'col': 'grupo_edad', 'mapping': {'0 a 6': 0,'7  a 11':1 ,'12 a 17': 2, '18 a 28': 3,'29 a 59':4,'60 y mas':5}},    {'col': 'ciclo_de_vida', 'mapping': {'Primera infancia': 0,  'Infancia': 1, 'Jovenes':2 ,'Adolescencia':3,'Adultez':4, 'Persona Mayor':5 }}]
from category_encoders import OrdinalEncoder
import category_encoders
encoder = OrdinalEncoder(mapping=mapping)

pipeline_ordinal = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('transformador_ordinal', category_encoders.ordinal.OrdinalEncoder(mapping=mapping))
])

pipeline_procesado = ColumnTransformer(
                   [('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), col_categoricas),
                    ('ordinal', pipeline_ordinal, col_ordinales)
                   ],
                remainder = 'passthrough',
                verbose_feature_names_out = False
               ).set_output(transform="pandas")



 Se establecen los datos de entrenamiento y de prueba para los modelos

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(variable_objetivo, axis=1), df[variable_objetivo], test_size=0.2, random_state=42)

In [7]:
X_train_prep = pipeline_procesado.fit_transform(X_train)
X_test_prep  = pipeline_procesado.transform(X_train)


In [8]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

### Se define la funcion `evaluar_modelo`, que se encarga de evaluar los diferentes modelos a partir de las metricas obtenidas

In [9]:
def evaluar_modelo(clases_reales, predicciones, probabilidades):
    exactitudes = []
    precisiones = []
    sensibilidades = []
    f1_scores = []
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_index, test_index in skf.split(X_train, y_train):
        # Se crea una copia del estimador del pipeline
        pipeline_estimador_copy = clone(pipeline_estimador)
        
        # Division de los datos en conjuntos de entrenamiento y prueba
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
        
        # Entrenamiento del modelo en el conjunto de entrenamiento actual
        pipeline_estimador_copy.fit(X_train_fold, y_train_fold)
        
        # Se obtienen las predicciones en el conjunto de prueba
        predicciones_fold = pipeline_estimador_copy.predict(X_test_fold)
        probabilidades_fold = pipeline_estimador_copy.predict_proba(X_test_fold)
        
        # Calculo de metricas
        exactitud = metrics.accuracy_score(y_test_fold, predicciones_fold)
        precision = metrics.precision_score(y_test_fold, predicciones_fold, average='macro', zero_division=0)
        sensibilidad = metrics.recall_score(y_test_fold, predicciones_fold, average='macro', zero_division=0)
        f1 = metrics.f1_score(y_test_fold, predicciones_fold, average='macro', zero_division=0)
        
        # Se guardan las metricas
        exactitudes.append(exactitud)
        precisiones.append(precision)
        sensibilidades.append(sensibilidad)
        f1_scores.append(f1)
    
    # Calculo de promedios para cada metrica
    exactitud_promedio = np.mean(exactitudes)
    precision_promedio = np.mean(precisiones)
    sensibilidad_promedio = np.mean(sensibilidades)
    f1_promedio = np.mean(f1_scores)
    
    print("""
    Exactitud promedio: {:.3f}
    Precisión promedio: {:.3f}
    Sensibilidad promedio: {:.3f}
    Puntuación F1 promedio: {:.3f}
    """.format(
        exactitud_promedio, 
        precision_promedio,
        sensibilidad_promedio,
        f1_promedio
    ))

<hr> 

# SEGUNDA PARTE

<hr> 

### Evaluación de parametros


A continuacion se presentan los mejores hiperparametros obtenidos del notebook de busqueda de hiperparametros para el modelo LightGBM

In [10]:
import lightgbm as lgb

In [11]:
# Exactitud promedio: 0.853

hiperparametros = {
    'learning_rate': 0.051417608292967786,
    'num_leaves': 26,
    'max_depth': 7,
    'min_child_samples': 62,
    'subsample': 0.7461116830392539,
    'colsample_bytree': 0.5904387690644886,
    'reg_alpha': 0.09233912822725567,
    'reg_lambda': 0.042356048609430104,
    'n_estimators': 126,
    'scale_pos_weight': 0.5842230278735413
}

In [17]:
# Exactitud promedio: 0.853

hiperparametros = {
    'lambda_l1': 0.9066726618630238, 
    'lambda_l2': 2.1912415916221266e-08, 
    'num_leaves': 13, 
    'feature_fraction': 0.6659436659143771, 
    'bagging_fraction': 0.8782998936356919, 
    'bagging_freq': 1, 
    'min_child_samples': 9,
    'learning_rate': 0.0593785754246979,
    'num_leaves': 20,
    'max_depth': 8,
    'min_child_samples': 30,
    'subsample': 0.563385117223741,
    'colsample_bytree': 0.5587125812504317,
    'reg_alpha': 0.0772254026786347,
    'reg_lambda': 0.025005311089717677,
    'n_estimators': 334,
    'scale_pos_weight': 7.646251254102079

}

In [23]:
# Exactitud promedio: 0.853

hiperparametros = {
    'boosting_type': 'dart', 
    'num_leaves': 968,
    'learning_rate': 0.04483452549986011,
    'n_estimators': 891, 
    'max_depth': 5,
    'min_child_samples': 4,
    'subsample': 0.8496110002568981,
    'colsample_bytree': 0.7324303718943765,
    'reg_alpha': 0.009677008893904163,
    'reg_lambda': 0.01017564831536729
}

In [29]:
# Exactitud promedio: 0.853

hiperparametros = {
    'num_leaves': 40,
    'n_estimators': 100,
    'min_child_samples': 10,
    'max_depth': 3,
    'learning_rate': 0.2
}

In [33]:
# Exactitud promedio: 0.851

hiperparametros = {
    'learning_rate': 0.011997488097121407, 
    'reg_alpha': 0.2423441665140374, 
    'reg_lambda': 0.001076864442889349, 
    'num_leaves': 158, 
    'max_depth': 5, 
    'min_child_samples': 8, 
    'subsample': 0.8165403953269705,
    'colsample_bytree': 0.8847367659829639,
    'n_estimators': 747, 
    'scale_pos_weight': 5.482662580491647
}

In [39]:
# Exactitud promedio: 0.851

hiperparametros = {
    'learning_rate': 0.01802606582997746, 
     'reg_alpha': 6.779674157856769e-08, 
     'reg_lambda': 0.053596159180180146, 
     'num_leaves': 23, 
     'max_depth': 7, 
     'min_child_samples': 4, 
     'subsample': 0.790561196083304, 
     'scale_pos_weight': 4.855801771130356, 
     'colsample_bytree': 0.972976079256426, 
     'n_estimators': 554
}


In [40]:
pipeline_estimador = Pipeline([
    ("procesado_variables", pipeline_procesado),
    ("estimador", lgb.LGBMClassifier(**hiperparametros))  
])

In [41]:
pipeline_estimador.fit(X=X_train, y=y_train)

Pipeline(steps=[('procesado_variables',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['nom_eve', 'area', 'comuna',
                                                   'paciente_hospitalizado',
                                                   'condicion_final',
                                                   'municipio', 'sexo', 'mes',
                                                   'departamento', 'actividad',
                                                   'año',
                                                   'violencia_intrafamiliar',
                                                   'escenario',
                                                   'agresor_menor_de_ed...
                                                  ['grupo_edad',
                                                   'ciclo_de_vida'])],
                                   verbose_feature_names_out=False)),
                ('estimador',
                 LGBMClassifier(colsample_bytree=0.972976079256426,
                                learning_rate=0.01802606582997746, max_depth=7,
                                min_child_samples=4, n_estimators=554,
                                num_leaves=23, reg_alpha=6.779674157856769e-08,
                                reg_lambda=0.053596159180180146,
                                scale_pos_weight=4.855801771130356,
                                subsample=0.790561196083304))])

In [42]:
predicciones =  pipeline_estimador.predict(X=X_test)
clases_reales = y_test
predicciones_probabilidades =pipeline_estimador.predict_proba(X_test)

In [43]:
evaluar_modelo(clases_reales, predicciones, predicciones_probabilidades)


    Exactitud promedio: 0.851
    Precisión promedio: 0.842
    Sensibilidad promedio: 0.841
    Puntuación F1 promedio: 0.841
    


In [44]:
from sklearn.metrics import accuracy_score
train_accuracy = accuracy_score(y_train, pipeline_estimador.predict(X_train))
test_accuracy = accuracy_score(y_test, pipeline_estimador.predict(X_test))
print("Accuracy en datos de entrenamiento:", train_accuracy)
print("Accuracy en datos de prueba:", test_accuracy)

Accuracy en datos de entrenamiento: 0.907425797197396
Accuracy en datos de prueba: 0.8521624007060901
